# rapyuta.impro
- plotrange
  + Btau
  + Ctau
  + [ellipse](#Ellipse)
  + SUE

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import sys
from pathlib import Path
import numpy as np

## Local
try:
    root = Path(__file__).parent.absolute() / '..'
except NameError:
    root = Path().absolute() / '..'
sys.path.insert(0, root)

import rapyuta.utbox as UT
import rapyuta.inout as IO
from rapyuta.inout import fitsext, h5ext
from rapyuta.plots import plotool, pplot
from rapyuta.impro import (
    improve, Jy_per_pix_to_MJy_per_sr,
    islice, icrop, irebin, igroupixel, ismooth,
    imontage, iswarp, iconvolve, cupid,
    wclean, interfill, hextract, hswarp,
    concatenate
)

%matplotlib inline

## fold inputs
# UT.codefold(True, 'onclick')


               \  \|/  /
            \ _\_ ||| _/_ /
                  |||
              /\/-/|\-\/\
          _ | _M_ _|_ _M_ | _
        / .~.    _ ^ _    .~. \
       |_|_0_|  |_&_&_|  |_0_|_|
      //    .~.    o    .~.    \\
     |_M_ _|_?_|__[_]__|_?_|_ _M_|
            |             |
             \  rapyuta  /
               \       /         _
    * ¨^  .~       =           ~^' _-
      ~ °

        Version 2.2.1 (20220616)




In [ ]:
## Set dir
datdir = root/'man/lib'
# outdir = root/'man/out'
outdir = root/'man/out/test_impro'
outdir.mkdir(parents=True, exist_ok=True)

datdir = str(datdir)
outdir = str(outdir)